# Combining Structured and UnStructured data


In [28]:
import openai
import os
from dotenv import load_dotenv

load_dotenv(".env", override=True)
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
!pip install llama-index==0.9.31  wikipedia

  Using cached llama_index-0.9.31-py3-none-any.whl (15.8 MB)
  Attempting uninstall: llama-index
    Found existing installation: llama-index 0.9.3
    Uninstalling llama-index-0.9.3:
      Successfully uninstalled llama-index-0.9.3


You should consider upgrading via the 'C:\llamaindex\llamaindex-samples\llamaindex\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,StorageContext,SQLDatabase
from llama_index.readers.wikipedia import WikipediaReader


### Create Common Objects

This includes a `ServiceContext` object containing abstractions such as the LLM and chunk size.
This also includes a `StorageContext` object containing our vector store abstractions.

In [29]:
from pinecone import Pinecone, PodSpec
from tqdm.autonotebook import tqdm

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
pc_index_name = "llamaindex-dochelper"
pinecone_index = pc.Index(pc_index_name)

In [8]:
# OPTIONAL: delete all
pinecone_index.delete(deleteAll=True)

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '55', 'x-pinecone-request-latency-ms': '13', 'x-pinecone-request-id': '3090145275427312615', 'date': 'Sun, 17 Mar 2024 16:35:24 GMT', 'x-envoy-upstream-service-time': '15', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":5,"message":"Namespace not found","details":[]}


In [30]:
from llama_index import ServiceContext
from llama_index.storage import StorageContext
from llama_index.vector_stores import PineconeVectorStore
from llama_index.node_parser import TokenTextSplitter
from llama_index.llms import OpenAI

# define node parser and LLM
chunk_size = 1024
llm = OpenAI(temperature=0, model="gpt-4", streaming=True)
service_context = ServiceContext.from_defaults(chunk_size=chunk_size, llm=llm)
node_parser = TokenTextSplitter(chunk_size=chunk_size)

# define pinecone vector index
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index, namespace="wiki_cities"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context)

### Create Database Schema + Test Data

Here we introduce a toy scenario where there are 100 tables (too big to fit into the prompt)

In [31]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [32]:
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [33]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [34]:
# print tables
metadata_obj.tables.keys()

dict_keys(['city_stats'])

We introduce some test data into the `city_stats` table

In [35]:
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [36]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Berlin', 3645000, 'Germany')]


### Load Data

We first show how to convert a Document into a set of Nodes, and insert into a DocumentStore.

In [16]:
# install wikipedia python package
!pip install wikipedia

You should consider upgrading via the 'C:\llamaindex\llamaindex-samples\llamaindex\Scripts\python.exe -m pip install --upgrade pip' command.


In [37]:
cities = ["Toronto", "Berlin", "Tokyo"]
wiki_docs = WikipediaReader().load_data(pages=cities)

### Build SQL Index

In [38]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [39]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

In [40]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
)

### Build Vector Index

In [41]:
# Insert documents into vector index
# Each document has metadata of the city attached
for city, wiki_doc in zip(cities, wiki_docs):
    nodes = node_parser.get_nodes_from_documents([wiki_doc])
    # add metadata to each node
    for node in nodes:
        node.metadata = {"title": city}
    vector_index.insert_nodes(nodes)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors: 100%|██████████| 18/18 [00:02<00:00,  8.19it/s]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors: 100%|██████████| 17/17 [00:02<00:00,  6.31it/s]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors: 100%|██████████| 11/11 [00:02<00:00,  4.53it/s]


### Define Query Engines, Set as Tools

In [42]:
from llama_index.query_engine import (
    SQLAutoVectorQueryEngine,
    RetrieverQueryEngine,
)
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.indices.vector_store import VectorIndexAutoRetriever

In [43]:
from llama_index.indices.vector_store.retrievers import (
    VectorIndexAutoRetriever,
)
from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo
from llama_index.query_engine.retriever_query_engine import (
    RetrieverQueryEngine,
)


vector_store_info = VectorStoreInfo(
    content_info="articles about different cities",
    metadata_info=[
        MetadataInfo(
            name="title", type="str", description="The name of the city"
        ),
    ],
)
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, service_context=service_context
)

In [44]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing: city_stats, containing the population/country of"
        " each city"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=(
        f"Useful for answering semantic questions about different cities"
    ),
)

### Define SQLAutoVectorQueryEngine

In [45]:
query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool, service_context=service_context
)

In [46]:
response = query_engine.query(
    "Tell me about the arts and culture of the city with the highest"
    " population"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Querying SQL database: The first choice is more relevant as it mentions 'city_stats' and 'population' which could be used to determine the city with the highest population. The second choice does not provide any specific information that could be used to answer the question.
INFO:llama_index.query_engine.sql_join_query_engine:> Querying SQL database: The first choice is more relevant as it mentions 'city_stats' and 'population' which could be used to determine the city with the highest population. The second choice does not provide any specific information that could be used to answer the question.
> Querying SQL database: The first choice is more relevant as it mentions 'city_stats' and 'population' which could be used to determine the city with the highest population. The second choice does not provide any specific 

In [47]:
print(str(response))

Tokyo, Japan, the city with the highest population, boasts a diverse and vibrant arts and culture scene. It is a blend of traditional Japanese arts like tea ceremonies and kabuki theater, and modern influences such as anime and manga. The city is home to numerous museums, galleries, and performance venues, showcasing a wide range of artistic expressions. Tokyo's neighborhoods each have their own unique character and offer a variety of cultural experiences.

Specifically, Tokyo houses the Tokyo National Museum, the country's largest museum specializing in traditional Japanese art. Other museums include the National Museum of Western Art, the Artizon Museum, the National Museum of Emerging Science and Innovation, the Edo-Tokyo Museum, the Nezu Museum, and the National Diet Library, National Archives, and the National Museum of Modern Art. 

For performing arts, there are national and private theaters for traditional forms of Japanese drama. The National Noh Theatre is dedicated to Noh, a

In [48]:
response = query_engine.query("Tell me about the history of Berlin")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Querying other query engine: The second choice seems more relevant to the question as it mentions answering semantic questions about different cities, which could include historical information. The first choice is more about translating queries into SQL, which doesn't seem directly related to the question about Berlin's history.
INFO:llama_index.query_engine.sql_join_query_engine:> Querying other query engine: The second choice seems more relevant to the question as it mentions answering semantic questions about different cities, which could include historical information. The first choice is more about translating queries into SQL, which doesn't seem directly related to the question about Berlin's history.
> Querying other query engine: The second choice seems more relevant to the question as it mentions answering s

In [49]:
print(str(response))

Berlin's history is rich and varied, dating back to around 9,000 BC with the Maglemosian culture. By 2,000 BC, dense human settlements had formed along the Spree and Havel rivers, giving rise to the Lusatian culture. Germanic tribes settled in the area around 500 BC, followed by the Burgundians after the Semnones left around 200 AD. In the 7th century, Slavic tribes reached the region.

In the 12th century, the region came under German rule as part of the Margraviate of Brandenburg, founded by Albert the Bear. The first written records of towns in the area of present-day Berlin date from the late 12th century. The two towns of Spandau and Köpenick formed close economic and social ties, and in 1307, they formed an alliance with a common external policy. The Hohenzollern family ruled in Berlin until 1918, first as electors of Brandenburg, then as kings of Prussia, and eventually as German emperors.

The Thirty Years' War between 1618 and 1648 devastated Berlin, with one third of its hous

In [50]:
response = query_engine.query(
    "Can you give me the country corresponding to each city?"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Querying SQL database: The first choice is most relevant as it mentions translating a natural language query into a SQL query over a table containing city_stats, which includes the country of each city. This aligns with the requirement of the question to provide the country corresponding to each city.
INFO:llama_index.query_engine.sql_join_query_engine:> Querying SQL database: The first choice is most relevant as it mentions translating a natural language query into a SQL query over a table containing city_stats, which includes the country of each city. This aligns with the requirement of the question to provide the country corresponding to each city.
> Querying SQL database: The first choice is most relevant as it mentions translating a natural language query into a SQL query over a table containing city_stats, which

In [51]:
print(str(response))

The corresponding countries for each city are Germany for Berlin, Japan for Tokyo, and Canada for Toronto.
